# **avaliar peso das palavras**

In [6]:
import joblib
import pandas as pd
import numpy as np

# 1. Carregar o arquivo pkl
# O joblib lê o dicionário que contém o vetorizador e o modelo
caminho_do_arquivo = 'modelo_b2w_rating_sentimento.pkl'
dados_do_pkl = joblib.load(caminho_do_arquivo)

def explicar_sentimento(texto, componentes, top_n=5):
    # Extrair os componentes de dentro do dicionário carregado
    vectorizer = componentes['vectorizer']
    classifier = componentes['model'] # Aqui estava o erro de 'cite' ou nomes trocados

    # 1. Transformar o texto usando o vetorizador treinado
    tfidf_matrix = vectorizer.transform([texto])
    feature_names = vectorizer.get_feature_names_out()

    # 2. Identificar quais palavras do texto estão no vocabulário do modelo
    feature_index = tfidf_matrix.nonzero()[1]

    if len(feature_index) == 0:
        return "Nenhuma palavra relevante encontrada para análise."

    data = []

    # 3. Calcular o impacto de cada palavra
    for idx in feature_index:
        palavra = feature_names[idx]
        valor_tfidf = tfidf_matrix[0, idx]

        # O .coef_[0] contém os pesos que o modelo dá a cada palavra
        peso_modelo = classifier.coef_[0][idx]

        impacto = valor_tfidf * peso_modelo

        data.append({
            'palavra': palavra,
            'impacto': impacto,
            'tendencia': 'Positivo' if impacto > 0 else 'Negativo'
        })

    # Criar DataFrame para organizar os resultados
    df = pd.DataFrame(data)

    # Ordenar pelo impacto absoluto (as palavras que mais "pesaram" na decisão)
    df['impacto_abs'] = df['impacto'].abs()
    resultado = df.sort_values(by='impacto_abs', ascending=False).head(top_n)

    return resultado[['palavra', 'tendencia', 'impacto']]

# ================= EXEMPLO DE USO =================

texto_para_testar = "O produto é excelente, mas a entrega atrasou um pouco."

# Chamamos a função passando o texto e o dicionário que carregamos no início
analise = explicar_sentimento(texto_para_testar, dados_do_pkl)

print(f"Análise do texto: '{texto_para_testar}'\n")
print(analise.to_string(index=False))

Análise do texto: 'O produto é excelente, mas a entrega atrasou um pouco.'

          palavra tendencia   impacto
        excelente  Negativo -1.272418
produto excelente  Negativo -0.348322
          atrasou  Negativo -0.311356
            pouco  Negativo -0.276649
          entrega  Negativo -0.177145


# **outra forma**

In [7]:
import joblib
import pandas as pd

# 1. Carregar o arquivo
dados_do_pkl = joblib.load('modelo_b2w_rating_sentimento.pkl')

def explicar_sentimento_detalhado(texto, componentes, top_n=5):
    vectorizer = componentes['vectorizer']
    classifier = componentes['model']

    # --- PARTE 1: Predição Geral e Probabilidade ---
    tfidf_matrix = vectorizer.transform([texto])

    # Predição da classe (ex: 0 para negativo, 1 para positivo)
    predicao = classifier.predict(tfidf_matrix)[0]

    # Cálculo da probabilidade
    probabilidades = classifier.predict_proba(tfidf_matrix)[0]
    prob_confianca = max(probabilidades) * 100 # Pega a maior probabilidade e vira %

    # Mapear o resultado (ajuste os nomes se o seu modelo usar 1/2 ou outros valores)
    sentimento_texto = "Positivo" if predicao == 1 else "Negativo"

    # --- PARTE 2: Impacto das Palavras (O "Porquê") ---
    feature_names = vectorizer.get_feature_names_out()
    feature_index = tfidf_matrix.nonzero()[1]

    data = []
    for idx in feature_index:
        palavra = feature_names[idx]
        peso = classifier.coef_[0][idx]
        valor_tfidf = tfidf_matrix[0, idx]
        impacto = valor_tfidf * peso

        data.append({
            'palavra': palavra,
            'impacto': impacto,
            'tendencia': 'Positivo' if impacto > 0 else 'Negativo'
        })

    df_palavras = pd.DataFrame(data)

    if not df_palavras.empty:
        df_palavras['impacto_abs'] = df_palavras['impacto'].abs()
        df_palavras = df_palavras.sort_values(by='impacto_abs', ascending=False).head(top_n)
        df_palavras = df_palavras[['palavra', 'tendencia', 'impacto']]

    return sentimento_texto, prob_confianca, df_palavras

# ================= TESTE COMPLETO =================

texto_teste = "O produto é maravilhoso, mas a entrega foi um desastre."

sentimento, confianca, detalhes = explicar_sentimento_detalhado(texto_teste, dados_do_pkl)

print("-" * 40)
print(f"TEXTO: {texto_teste}")
print("-" * 40)
print(f"SENTIMENTO PREVISTO: {sentimento}")
print(f"CONFIANÇA DO MODELO: {confianca:.2f}%")
print("-" * 40)
print("PALAVRAS QUE MAIS INFLUENCIARAM:")
print(detalhes.to_string(index=False))
print("-" * 40)

----------------------------------------
TEXTO: O produto é maravilhoso, mas a entrega foi um desastre.
----------------------------------------
SENTIMENTO PREVISTO: Negativo
CONFIANÇA DO MODELO: 94.62%
----------------------------------------
PALAVRAS QUE MAIS INFLUENCIARAM:
            palavra tendencia   impacto
        maravilhoso  Negativo -1.005276
produto maravilhoso  Negativo -0.332598
            entrega  Negativo -0.249232
            produto  Positivo  0.215194
maravilhoso entrega  Negativo -0.084287
----------------------------------------
